<a href="https://colab.research.google.com/github/Bustagar/PROCESAMIENTO-DE-HABLA/blob/main/TP4_chatbot(Resuelto).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP4 Chatbot

## Nombre: Juan Sebastian Bustamante Garcia

In [ ]:
!pip install spacy --quiet
!python -m spacy download es_core_news_sm --quiet


In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")
import es_core_news_sm
nlp = es_core_news_sm.load()
doc = nlp("Esto es una frase.")
print([(w.text, w.pos_) for w in doc])

# Chatbots basados en recuperación

En inglés information retrieval chatbots

# Motor de búsqueda

* Búsqueda por palabras clave: Extrae palabras clave de la pregunta del usuario y busca coincidencias en las preguntas almacenadas.

* Similitud del coseno: Si has representado las preguntas como vectores (por ejemplo, usando TF-IDF o word embeddings), puedes usar la similitud del coseno para medir la distancia entre las preguntas.

* Word embeddings: Utiliza modelos de word embeddings como Word2Vec o BERT para obtener representaciones semánticas de las preguntas y las consultas del usuario.

### Búsqueda por palabras claves

In [ ]:
tu_diccionario = {
   "hola": "¡Hola! ¿En qué puedo ayudarte?",
   "adiós": "Hasta luego. ¡Que tengas un buen día!",
   "información": "¿Qué tipo de información estás buscando?",
   # Agrega más entradas de diccionario según tus necesidades
}


In [ ]:
def responder_pregunta(pregunta):
    pregunta_procesada = nlp(pregunta.lower())  # Procesa la pregunta y convierte a minúsculas
    respuesta = "Lo siento, no entiendo tu pregunta."

    # Busca una coincidencia en el diccionario
    for palabra in pregunta_procesada:
        # regresa la primer coincidencia que encuentra
        if palabra.text in tu_diccionario:
            respuesta = tu_diccionario[palabra.text]
            break

    return respuesta


In [ ]:
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot: Hasta luego.")
        break
    respuesta = responder_pregunta(entrada_usuario)
    print("Chatbot:", respuesta)


## Búsqueda por similitud

Para los chatbots basados ​​en recuperación, es común utilizar bolsas de palabras (bag of words) o tf-idf para calcular la similitud de intenciones.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Datos de ejemplo
preguntas = ["¿Qué es el aprendizaje automático?",
             "¿Cómo funciona la regresión lineal?"]
respuestas = ["El aprendizaje automático es una rama de la inteligencia artificial...",
              "La regresión lineal es un método de modelado..."]

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preguntas)

# Función para encontrar la mejor coincidencia
def responder_pregunta(consulta_usuario):
    consulta_vec = vectorizer.transform([consulta_usuario])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    print(similitudes)
    indice_mejor_coincidencia = similitudes.argmax()
    print(indice_mejor_coincidencia)
    return respuestas[indice_mejor_coincidencia]


In [ ]:

# Ejemplo de consulta
consulta = "¿Qué es la regresión lineal?"
print(responder_pregunta(consulta))


## Búsqueda por similitud en embeddings

Puedes vectorizar el texto usando embeddings, como vimos la clase pasada.


## Actividades

### 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular. ( 3 puntos )

1.1 Debe definir la aplicación (atención al cliente bancario, atención a estudiantes universitarios, etc).
1.2 El listado de preguntas y respuestas debe tener como mínimo 20 elementos pregunta - respuesta.

###  2) Crear el chatbot utilizando TFIDF y similitud del coseno. (1 punto)

### 3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

### 4) Muestra ambos chatbots funcionando (1 punto)

Adjuntar la lista de preguntas utilizadas para probar el funcionamiento.

### 5) Añade tus conclusiones de todo lo realizado (2 punto)

### 6) BONUS: usa lo realizado en 1 y 3 para crear un chatbot RAG. (2 puntos)

* Utiliza un modelo LLM pre-entrenado.

* Este punto no es obligatorio de realizar para quienes quieran regularizar / recuperar y luego rendirán en mesa.
* Para quienes tienen condiciones para promocionar (han realizado y entregado los TPs a tiempo) la resolución de este ejercicio será tenida en cuenta para sumar a la promoción.

### 7) No olvides:

* Explicar tus decisiones y configuraciones. Añadir tus conclusiones.
* Anunciar en el foro cuál será tu aplicación y postear tu entrega y tus avances.
* Debes subir tu notebook a un repo GitHub público de tu propiedad compartido + enlace colab.
* Documentar todo el proceso.





## 1. Elección de aplicación (Chatbot Jurídico)

In [ ]:
!pip install spacy --quiet
!python -m spacy download es_core_news_sm --quiet

In [ ]:
import pandas as pd
import nltk
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Carga del dataset
url = 'https://raw.githubusercontent.com/Bustagar/PROCESAMIENTO-DE-HABLA/refs/heads/main/Preguntas_Derecho.csv'

df = pd.read_csv(url)
df.head()

In [ ]:
# Carga del modelo de spaCy y descarga de recursos
nlp = spacy.load("es_core_news_sm")
nltk.download('stopwords')
stopwords_es = set(nltk.corpus.stopwords.words('spanish'))

## 2.1 Procesamiento y limpieza

In [ ]:
def limpiar_texto(text):
    text = text.lower()                                    # Convertir todo a minúsculas
    text = re.sub(r'[.¡!¿?\'\"“”‘’«»…]', '', text)         # Eliminar signos de puntuación comunes
    text = re.sub(r'\s+', ' ', text).strip()               # Reemplazar múltiples espacios por uno
    text = re.sub(r'\.\.\.', '', text)                     # Eliminar puntos suspensivos
    text = re.sub(r'[^a-zñáéíóúü\s]', '', text)            # Quitar todo lo que no sea letra o espacio
    return text

def lematizar_y_filtrar(text):
    doc = nlp(text)                                        # Procesar el texto con spaCy
    lemmas = [token.lemma_ for token in doc
              if token.lemma_ not in stopwords_es and token.is_alpha]  # Lemas sin stopwords
    return " ".join(lemmas)                                # Unir los lemas en una sola cadena

def procesar_preguntas(preguntas):
    preguntas_limpias = [limpiar_texto(q) for q in preguntas]           # Aplicar limpieza básica a cada pregunta
    preguntas_procesadas = [lematizar_y_filtrar(q) for q in preguntas_limpias]  # Lematizar y eliminar stopwords
    return preguntas_procesadas

- Se lematizaron primeramente las preguntas antes de quitar las stopword ya que al haber muchas palabras conjugadas o flexionadas no las reconoce el stopword y las deja en el texto, produciendo mas ruido.

## 2.2 Vectorizacion aplicando TF-IDF

In [ ]:
preguntas_procesadas = procesar_preguntas(df['Pregunta'].tolist())  # Procesar todas las preguntas

df['Pregunta_procesada'] = preguntas_procesadas           # Agregar columna con preguntas procesadas al DataFrame

vectorizer = TfidfVectorizer()                            # Crear el vectorizador TF-IDF
X = vectorizer.fit_transform(df['Pregunta_procesada'])    # Vectorizar las preguntas del corpus

## 2.3 Chatbot con TF-IDF

In [ ]:
def chatbot_responder(pregunta_usuario):
    entrada = pregunta_usuario.lower().strip()

    # Respuesta personalizada si detecta un saludo simple
    if entrada in ['hola', 'buenas', 'buen día', 'buenas tardes', 'buenas noches', 'qué tal', 'saludos']:
        return "Hola! ¿En qué puedo ayudarte?"

    pregunta_proc = lematizar_y_filtrar(limpiar_texto(entrada)) # Procesar la entrada del usuario
    vector_usuario = vectorizer.transform([pregunta_proc])      # Vectorizar la pregunta procesada
    similitudes = cosine_similarity(vector_usuario, X)          # Calcular similitud entre usuario y corpus

    idx_max = similitudes.argmax()                   # Obtener el índice de la mejor coincidencia
    score = similitudes[0][idx_max]                  # Obtener el puntaje de similitud más alto

    if score > 0.3:                                  # Si el puntaje es razonable
        return df.iloc[idx_max]['Respuesta']         # Devolver la respuesta correspondiente
    else:
        return "No tengo una respuesta segura. ¿Podrías reformular la pregunta?"  # Mensaje por defecto


## Evaluación de la similitud del coseno

In [ ]:
def evaluar_similitud(pregunta_usuario):
    # Preprocesar la entrada
    pregunta_proc = lematizar_y_filtrar(limpiar_texto(pregunta_usuario))

    # Vectorizar
    pregunta_vec = vectorizer.transform([pregunta_proc])

    # Calcular similitud con todas las preguntas del corpus
    similitudes = cosine_similarity(pregunta_vec, X)

    # Obtener índice y puntaje de la mejor coincidencia
    idx_max = similitudes.argmax()
    score_max = similitudes[0][idx_max]

    # Respuesta asociada
    respuesta = df.iloc[idx_max]['Respuesta']

    # Imprimir detalle
    print(f"Pregunta procesada:  {pregunta_proc}")
    print(f"Similitud máxima:    {score_max:.4f}")
    print(f"Índice en dataset:   {idx_max}")
    print(f"Respuesta sugerida:  {respuesta}")

    return score_max, idx_max, respuesta


In [ ]:
# Se prueba la similitud
pregunta = "Delito doloso"
print(evaluar_similitud(pregunta))


## Se observa que al evaluar el indice del coseno se obtiene una similitud del 99% lo cual sugiere que el modelo capta muy bien la similitud

# 3. Crear otro chatbot utilizando embeddings

## MiniLm: Se utilizó este modelo ya que es liviano y multilingual y muy eficiente para medir similitud semántica entre frases, lo cual encontré que es ideal para lenguaje Jurídico aunque hay un modelo medianamente entrenado con lenguaje juríco que es Roberta, pero este embeddings no esta listo para tareas semánticas e iba a tener que hacerlo manualmente

In [ ]:
# Importación de librerias
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

## 3.1. Carga de modelo y codificación de preguntas

In [ ]:
# Cargar modelo
modelo_embedding = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Codificar preguntas del dataset (evitar nulos)
preguntas_corpus = df['Pregunta'].fillna('').tolist()

# Codificar todas las preguntas del corpus en vectores numéricos
embeddings_corpus = modelo_embedding.encode(preguntas_corpus, convert_to_tensor=True)

## 3.2. Chatbot con embeddings MiniLm

In [ ]:
def chatbot_responder_minilm(pregunta_usuario):
    entrada = pregunta_usuario.lower().strip()

    # Detectar saludo básico
    if entrada in ['hola', 'buenas', 'buen día', 'buenas tardes', 'buenas noches', 'qué tal', 'saludos']:
        return "Hola! ¿En qué puedo ayudarte?"

    # Codificar la pregunta con embeddings MiniLM
    pregunta_emb = modelo_embedding.encode(pregunta_usuario, convert_to_tensor=True)
    similitudes = cos_sim(pregunta_emb, embeddings_corpus)[0]

    # Mejor coincidencia
    idx_max = similitudes.argmax().item()
    score_max = similitudes[idx_max].item()

    # Evaluar umbral de similitud
    if score_max > 0.5:
        return df.iloc[idx_max]['Respuesta']
    else:
        return "Lo siento, no encuentro una respuesta clara. ¿Querés reformular la pregunta?"




## 3.3 Evaluación similitud del coseno

In [ ]:
def evaluar_similitud_minilm(pregunta_usuario):
    # Codificar la pregunta como embedding con MiniLM
    pregunta_emb = modelo_embedding.encode(pregunta_usuario, convert_to_tensor=True)

    # Calcular similitud del coseno con todo el corpus embebido
    similitudes = cos_sim(pregunta_emb, embeddings_corpus)[0].cpu().numpy()

    # Obtener índice y puntaje de la mejor coincidencia
    idx_max = similitudes.argmax()
    score_max = similitudes[idx_max]

    # Respuesta asociada
    respuesta = df.iloc[idx_max]['Respuesta']

    # Imprimir detalle
    print(f"Pregunta original:    {pregunta_usuario}")
    print(f"Similitud máxima:     {score_max:.4f}")
    print(f"Índice en dataset:    {idx_max}")
    print(f"Pregunta coincidente: {df.iloc[idx_max]['Pregunta']}")
    print(f"Respuesta sugerida:   {respuesta}")

    return score_max, idx_max, respuesta


In [ ]:
# Se prueba la similitud
pregunta = "Delito doloso"
print(evaluar_similitud(pregunta))

- Se observa una similitud bastante baja del 99%, lo que sugiere que el embeddings encuentra una buena similitud con respecto a la pregunta usada por el usuario y la contenida en el dataset.



# 4. Muestra ambos chatbots funcionando

## 4.1. Lista de preguntas reformuladas para ver la similitud

In [ ]:
preguntas_test = [
    "¿Cómo se define el Derecho Constitucional?",
    "¿Para qué sirve la Constitución Nacional?",
    "¿Quién se encarga de supervisar si una ley es constitucional?",
    "¿Qué tipo de derechos están garantizados en la Constitución?",
]


## 4.2 Prueba del Chatbot con TF-IDF

In [ ]:
while True:
    entrada = input("Tú: ")
    if entrada.lower() in ['salir', 'exit', 'adiós', 'adios', 'chau', 'chao', 'nos vemos', 'gracias']:
        print("\nChatbot: ¡Hasta luego!")
        break

    respuesta = chatbot_responder(entrada)
    print(f"\nChatbot: {respuesta}\n")

# 4.3 Prueba del Chatbot con Embeddings

In [ ]:
while True:
    entrada = input("Tú: ")
    if entrada.lower() in ['salir', 'exit', 'adiós', 'adios', 'chau', 'chao', 'nos vemos', 'gracias']:
        print("\nChatbot: ¡Hasta luego!")
        break

    respuesta = chatbot_responder_minilm(entrada)
    print(f"\nChatbot: {respuesta}\n")